# Import

In [1]:
import time
from os import listdir
import os
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
device = torch.device("cuda")
from transformers import*

I1205 05:30:24.063823 139765610493696 file_utils.py:39] PyTorch version 1.1.0 available.


# Load data & Preprocess

In [2]:
def to_input_format(query, doc):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    query_tokens = tokenizer.tokenize(query)
    doc_tokens = tokenizer.tokenize(doc)
    input_tokens = [cls_token] + query_tokens + [sep_token] + doc_tokens
    input_ids = tokenizer.convert_tokens_to_ids(input_tokens)[:512]
    token_type = ([0] * (2+len(query_tokens)) + [1] * (len(doc_tokens)))[:512]
    attention_mask = [1] * len(input_ids)
    if len(input_ids)<512:
        pad = 512 - len(input_ids)
        input_ids = input_ids + [0] * pad
        token_type = token_type + [2] * pad
        attention_mask = attention_mask + [0] * pad
    return np.array(input_ids), np.array(token_type), np.array(attention_mask)

In [3]:
def load_training_data(train_df):
    train_x = []
    train_y = []
    for idx,row in train_df.iterrows():
        fq = open("train/query/"+str(row.query), 'r') 
        qry = fq.read()
        fd = open("doc/"+str(row.document), 'r') 
        doc = fd.read()
        train_x.append(to_input_format(qry, doc))
        train_y.append(row.label)  
    return train_x, train_y

In [4]:
def load_test_data(qry_file_name, doc_file_name):
    fq = open("test/query/" + qry_file_name, 'r') 
    qry = fq.read()
    fd = open("doc/" + doc_file_name, 'r') 
    doc = fd.read()
    test_x = to_input_format(qry, doc)
    return test_x

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', max_length=512)

# training data 
pos = pd.read_csv('train/Pos.txt', sep=" ", header=None)
neg = pd.read_csv('train/Neg.txt', sep=" ", header=None)
train_df = pd.concat([pos,neg])
train_df.columns = ['query','document','label']
train_x, train_y = load_training_data(train_df[:10])

train_x = torch.tensor(train_x, dtype=torch.long)
train_y = torch.tensor(train_y, dtype=torch.float)

# testing data file name list
docs = listdir('doc') 
qrys = listdir("test/query")

I1205 05:30:25.245392 139765610493696 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/deep3072/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


# Build Model

In [6]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)

I1205 05:30:26.477765 139765610493696 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/deep3072/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I1205 05:30:26.479765 139765610493696 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 305

# Train

In [7]:
model.to(device)
# model.load_state_dict(torch.load('model.pkl'))
model.train()
optimizer = optim.Adam(model.parameters(), lr=2e-5)
optimizer.zero_grad()
criterion = nn.BCEWithLogitsLoss()

In [8]:
for epoch in range(1):   
    print("epoch:",epoch)
    l=0
    for i in range(len(train_x)): 
        
        input_ids, token_type_ids, attention_mask= train_x[i]
        labels = train_y[i]
        input_ids = input_ids.cuda()
        token_type_ids = token_type_ids.cuda()
        attention_mask = attention_mask.cuda()
        labels = labels.cuda()
        _,outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, labels=labels)
        
        loss = criterion(outputs.squeeze(-1), labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    print(l/step)
    torch.save(model.state_dict(), 'model.pkl')

epoch: 0


UnboundLocalError: local variable 'extended_attention_mask' referenced before assignment

# Test

In [9]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_aff

In [10]:
fname = "./submission.txt"
f = open(fname, 'w')
f.write("Query,RetrievedDocuments\n")  

for q in range(len(qrys)): 
    qry_file_name=qrys[q]
    f.write(qry_file_name + ",") 
    sim = np.zeros(len(docs))
    for j in range(len(docs)):
        doc_file_name=docs[j]
        test_x = load_test_data(qry_file_name, doc_file_name)
        test_x = torch.tensor(test_x, dtype=torch.long)
        input_ids, token_type_ids, attention_mask = test_x
        input_ids = input_ids.cuda()
        token_type_ids = token_type_ids.cuda()
        attention_mask = attention_mask.cuda()
        _,outputs = model(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        sim[j] = outputs.squeeze(-1)
    rank = np.argsort(-sim)
    for j in rank[:100]:
        f.write(docs[j]+" ")
    f.write("\n")

UnboundLocalError: local variable 'extended_attention_mask' referenced before assignment

# for one input

In [11]:
q = "I am fool"
d = "hello how are you , kekkek mekwjsa ,edas,dl"

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
cls_token = tokenizer.cls_token
sep_token = tokenizer.sep_token

q = f"{cls_token} {q} {sep_token}"
print(q)

query = tokenizer.tokenize(q)
query_ids = tokenizer.convert_tokens_to_ids(query)
print(query_ids)

doc_len = 512-len(query_ids)
doc = tokenizer.tokenize(d)
doc_ids = tokenizer.convert_tokens_to_ids(doc)
doc_ids = doc_ids[:doc_len]
print(doc_ids)

ids = query_ids + doc_ids
ids = torch.LongTensor([ids])

attention_mask = [*(1 for _ in query_ids), *(1 for _ in doc_ids)]
attention_mask = torch.LongTensor([attention_mask])

labels = 1
labels = torch.FloatTensor([labels])

batch = (ids, attention_mask, labels)

I1205 05:30:53.707674 139765610493696 tokenization_utils.py:375] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/deep3072/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


[CLS] I am fool [SEP]
[101, 1045, 2572, 7966, 102]
[7592, 2129, 2024, 2017, 1010, 17710, 19658, 5937, 2033, 2243, 2860, 22578, 2050, 1010, 3968, 3022, 1010, 21469]


In [12]:
bert_finetune = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
optimizer = optim.Adam(bert_finetune.parameters(), lr=2e-5)
optimizer.zero_grad()
criterion = nn.BCEWithLogitsLoss()
bert_finetune.to(device)
bert_finetune.train()
ids = ids.cuda()
attention_mask = attention_mask.cuda()
labels = labels.cuda()

_, outputs = bert_finetune(input_ids=ids, attention_mask=attention_mask, labels=labels)
loss = criterion(outputs.squeeze(-1), labels)
loss.backward()
optimizer.step()
optimizer.zero_grad()
print(loss)
print(outputs)

I1205 05:30:55.975672 139765610493696 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/deep3072/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I1205 05:30:55.979140 139765610493696 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 1,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 305

tensor(0.7456, device='cuda:0', grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
tensor([[-0.1023]], device='cuda:0', grad_fn=<AddmmBackward>)
